# Imports and Session

In [1]:
# find spark content
import findspark
findspark.init()

import numpy as np
import pandas as pd
import datetime
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, desc 
from pyspark.sql.functions import sum as scala_sum, unix_timestamp 
from pyspark.sql.functions import from_unixtime, col
from pyspark.sql.types import DateType, IntegerType, StringType, BooleanType, TimestampType
from pyspark.sql import Window

In [2]:
# create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset

In [3]:
# Read in full sparkify dataset
event_data = "mini_sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

# Pre-processing

## Cleaning Date

In [4]:
# date cleanup
to_datetime = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime("%Y-%m-%d"))
df = df.withColumn("date", to_datetime(df.ts))
df.select('date').show()

+----------+
|      date|
+----------+
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
|2018-09-30|
+----------+
only showing top 20 rows



In [5]:
# datetime iso format cleanup
to_datetime = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).isoformat())
df = df.withColumn("datetime", to_datetime(df.ts))
df.select('datetime').show()

+-------------------+
|           datetime|
+-------------------+
|2018-09-30T20:01:57|
|2018-09-30T20:03:00|
|2018-09-30T20:06:34|
|2018-09-30T20:06:56|
|2018-09-30T20:11:16|
|2018-09-30T20:11:18|
|2018-09-30T20:14:46|
|2018-09-30T20:14:59|
|2018-09-30T20:15:05|
|2018-09-30T20:18:04|
|2018-09-30T20:19:06|
|2018-09-30T20:19:10|
|2018-09-30T20:20:18|
|2018-09-30T20:22:55|
|2018-09-30T20:22:56|
|2018-09-30T20:24:01|
|2018-09-30T20:26:16|
|2018-09-30T20:27:48|
|2018-09-30T20:28:07|
|2018-09-30T20:29:04|
+-------------------+
only showing top 20 rows



In [6]:
# pandas dataframe to explore
df_explore = df.sample(True, 0.30, 42).toPandas()

## First 30 Unique Values

In [7]:
# check first 30 unique values
for column in df_explore:
    print(column, "=", df_explore[column].unique()[:30], "\n"+"- -"*25)

artist = ['The All-American Rejects' 'The Velvet Underground / Nico' 'Tesla'
 'Florence + The Machine' 'Manolo Garcia' 'Downhere' 'Modjo'
 'Kings Of Leon' 'Nine Inch Nails' 'Campo' 'Kix'
 'Lambert_ Hendricks & Ross' 'Kanye West' 'Eminem' 'Charttraxx Karaoke'
 'Hate Eternal' 'Scars On Broadway' 'The Last Goodnight' None
 'BjÃ\x83Â¶rk' 'Boston' 'Evan Rachel Wood' 'Parachute' 'Nevermore'
 'Sea Wolf' 'Beyonce & Shakira' 'Traveling Wilburys' 'En Vogue'
 'Galaxie 500' 'Moose'] 
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
auth = ['Logged In' 'Logged Out' 'Cancelled' 'Guest'] 
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
firstName = ['Micah' 'Ashlynn' 'Colin' 'Alexi' 'Warren' None 'Ainsley' 'Alex' 'Zyonna'
 'Faigy' 'Madison' 'Grant' 'Anika' 'Isobella' 'Payton' 'Teagan'
 'Christian' 'Kael' 'Daelin' 'Giovanni' 'Jaleel' 'Panav' 'Carly'
 'Elizabeth' 'Deegan' 'Riley' 'Ethan' 'Molly' 'Bryson' 'Nicole'] 
- -- -- -- -- -- -- -- -- -- -- --

### Last 3 Digits in ts

In [8]:
# validating last 3 digits being zero
last_3_func = udf(lambda x: x[-3:], StringType())
df.withColumn("last3", last_3_func(df.ts.cast('string'))).groupBy("last3").count().show()

+-----+------+
|last3| count|
+-----+------+
|  000|286500|
+-----+------+



---

## Cancellations

### Autorization and Page Counts

In [9]:
df.groupBy(['auth', 'page']).count().show(100)

+----------+--------------------+------+
|      auth|                page| count|
+----------+--------------------+------+
|Logged Out|                Home|  4339|
| Logged In|              Logout|  3226|
|     Guest|            Register|    18|
| Logged In|           Thumbs Up| 12551|
|Logged Out|               Error|     5|
|     Guest|               About|    14|
| Logged In|         Roll Advert|  3933|
|     Guest|               Error|     1|
| Logged In|      Submit Upgrade|   159|
|Logged Out|               Login|  3241|
| Logged In|            Settings|  1514|
| Logged In|         Thumbs Down|  2546|
|Logged Out|                Help|   249|
| Logged In|       Save Settings|   310|
| Logged In|                Help|  1454|
| Logged In|            NextSong|228108|
| Logged In|               About|   495|
|     Guest| Submit Registration|     5|
| Logged In|     Add to Playlist|  6526|
| Logged In|               Error|   252|
|Logged Out|               About|   415|
| Logged In|    

> All cancellation confirmations are cancelled.

### Applying Cancellation Phase

In [10]:
# window function
window_fun = (Window.partitionBy("userId")
             .orderBy(desc("ts"))
             .rangeBetween(Window.unboundedPreceding, 0))

subset = ['userId', "datetime", "page", "level"]

In [11]:
# flagging downgrades
flag_cancel_func = udf(lambda x: 1 if x == "Cancellation Confirmation"
                           else 0, IntegerType())
# adding indicator
df = df.withColumn("cancel_ind", flag_cancel_func("page"))

# sum flags over the window statement
df = df.withColumn("cancel_user", 
                   scala_sum("cancel_ind").over(window_fun))

df.head()

Row(artist=None, auth='Logged In', firstName='Darianna', gender='F', itemInSession=34, lastName='Carpenter', length=None, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='Logout', registration=1538016340000, sessionId=187, song=None, status=307, ts=1542823952000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', date='2018-11-21', datetime='2018-11-21T13:12:32', cancel_ind=0, cancel_user=0)

### Cancelled Users

In [12]:
df.filter(df.cancel_ind == 1).select('userId').distinct().show(100)

+------+
|userId|
+------+
|   125|
|    51|
|    54|
|100014|
|   101|
|    29|
|100021|
|    87|
|    73|
|     3|
|    28|
|100022|
|100025|
|300007|
|100006|
|    18|
|    70|
|100005|
|    17|
|100007|
|300001|
|100009|
|100015|
|200024|
|100003|
|   103|
|100024|
|    53|
|   122|
|200017|
|    58|
|100011|
|100019|
|100012|
|200018|
|200016|
|200020|
|   106|
|   143|
|    32|
|200001|
|   105|
|200011|
|100023|
|100013|
|100017|
|   121|
|    12|
|200015|
|   129|
|200021|
|100001|
+------+



### Example of Cancellation Phase of User

In [13]:
cancel_subset = ['userId', "datetime", 
                 "page", "level", "cancel_user"]

df_user = df.select(cancel_subset).where(
    df.userId == '100001').orderBy(cancel_subset)

In [14]:
print(f"There are {df_user.count():2,.0f} transcactions")
df_user.show(df_user.count())

There are 187 transcactions
+------+-------------------+--------------------+-----+-----------+
|userId|           datetime|                page|level|cancel_user|
+------+-------------------+--------------------+-----+-----------+
|100001|2018-10-01T02:48:24|                Home| free|          1|
|100001|2018-10-01T02:48:29|            NextSong| free|          1|
|100001|2018-10-01T02:49:02|         Roll Advert| free|          1|
|100001|2018-10-01T02:52:27|            NextSong| free|          1|
|100001|2018-10-01T02:53:03|         Roll Advert| free|          1|
|100001|2018-10-01T03:02:29|            NextSong| free|          1|
|100001|2018-10-01T03:09:08|            NextSong| free|          1|
|100001|2018-10-01T03:12:12|            NextSong| free|          1|
|100001|2018-10-01T03:17:25|            NextSong| free|          1|
|100001|2018-10-01T03:21:23|            NextSong| free|          1|
|100001|2018-10-01T03:24:47|            NextSong| free|          1|
|100001|2018-10-01T0

---

## Downgrades

### Applying Downgrade Phase

In [15]:
# flagging downgrades
flag_downgraded_func = udf(lambda x: 1 if x == "Submit Downgrade"
                           else 0, IntegerType())

# adding indicator
df = df.withColumn("downgrade_ind", flag_downgraded_func("page"))

# sum flags over the window statement
df = df.withColumn("downgrade_phase", 
                   scala_sum("downgrade_ind").over(window_fun))

df.head()

Row(artist=None, auth='Logged In', firstName='Darianna', gender='F', itemInSession=34, lastName='Carpenter', length=None, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='Logout', registration=1538016340000, sessionId=187, song=None, status=307, ts=1542823952000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', date='2018-11-21', datetime='2018-11-21T13:12:32', cancel_ind=0, cancel_user=0, downgrade_ind=0, downgrade_phase=0)

### Downgrade Users

In [16]:
df.filter(df.downgrade_ind == 1).select('userId').distinct().show(100)

+------+
|userId|
+------+
|    54|
|    11|
|    30|
|    59|
|    85|
|    35|
|100008|
|100025|
|300002|
|    96|
|   100|
|    61|
|   131|
|   140|
|300004|
|300015|
|    77|
|100009|
|300011|
|100004|
|200003|
|100015|
|300023|
|100016|
|    95|
|    38|
|   103|
|    25|
|200009|
|200023|
|    92|
|    81|
|100012|
|200025|
|200020|
|   141|
|     9|
|    24|
|    20|
|200011|
|    49|
|300021|
|    39|
|    12|
|   109|
|    13|
|200019|
|    74|
|100018|
+------+



### Example of Downgrade Phase of User

In [17]:
downgrade_subset = ["datetime", 
                 "page", "level", "downgrade_phase", "cancel_user"]

df_user = df.select(downgrade_subset).where(
    df.userId == '200011').orderBy(downgrade_subset)

In [18]:
print(f"There are {df_user.count():2,.0f} transcactions")
df_user.show(df_user.count())

There are 848 transcactions
+-------------------+--------------------+-----+---------------+-----------+
|           datetime|                page|level|downgrade_phase|cancel_user|
+-------------------+--------------------+-----+---------------+-----------+
|2018-10-02T08:43:21|            NextSong| free|              1|          1|
|2018-10-02T08:47:27|            NextSong| free|              1|          1|
|2018-10-02T08:47:31|                Home| free|              1|          1|
|2018-10-02T08:51:19|            NextSong| free|              1|          1|
|2018-10-02T08:51:52|         Roll Advert| free|              1|          1|
|2018-10-02T08:55:12|            NextSong| free|              1|          1|
|2018-10-02T08:59:44|            NextSong| free|              1|          1|
|2018-10-02T09:00:50|                Help| free|              1|          1|
|2018-10-02T09:01:07|                Home| free|              1|          1|
|2018-10-02T09:04:22|            NextSong| free|

In [20]:
df.head()

Row(artist=None, auth='Logged In', firstName='Darianna', gender='F', itemInSession=34, lastName='Carpenter', length=None, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='Logout', registration=1538016340000, sessionId=187, song=None, status=307, ts=1542823952000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', date='2018-11-21', datetime='2018-11-21T13:12:32', cancel_ind=0, cancel_user=0, downgrade_ind=0, downgrade_phase=0)

## Period Between Downgrade

In [31]:
df.createOrReplaceTempView("sparkify_table")

### Period Spark Version

In [126]:
period_function = udf(lambda page: int(page=='Downgrade'), IntegerType())

over_func = (Window.partitionBy("userId")
    .orderBy(desc("ts"))
    .rangeBetween(Window.unboundedPreceding, 0))

(df.filter((df.page == 'NextSong') | (df.page == 'Downgrade'))
    .select(["userId", "page", "gender", "length", "ts"])
    .withColumn("downgrade_ind", period_function(col("page")))
    .withColumn("period", scala_sum("downgrade_ind").over(over_func))
).filter(df.userId == '200011').show(100)

+------+---------+------+---------+-------------+-------------+------+
|userId|     page|gender|   length|           ts|downgrade_ind|period|
+------+---------+------+---------+-------------+-------------+------+
|200011| NextSong|     M|335.90812|1541120924000|            0|     0|
|200011| NextSong|     M|220.86485|1541120704000|            0|     0|
|200011| NextSong|     M|311.11791|1541120393000|            0|     0|
|200011| NextSong|     M|249.93914|1541120144000|            0|     0|
|200011| NextSong|     M|201.79546|1541119943000|            0|     0|
|200011| NextSong|     M|219.66322|1541010279000|            0|     0|
|200011| NextSong|     M|518.81751|1541009761000|            0|     0|
|200011| NextSong|     M|253.98812|1541009508000|            0|     0|
|200011| NextSong|     M|187.76771|1541009321000|            0|     0|
|200011| NextSong|     M|141.11302|1541009180000|            0|     0|
|200011| NextSong|     M|125.33506|1541009055000|            0|     0|
|20001

### Period SQL Version

In [121]:
downgrade_indicator = spark.sql("""
SELECT 
    userId, 
    page, 
    gender,
    length,
    date,
    ts,
    CASE 
        WHEN page = 'Downgrade' 
        THEN 1 
        ELSE 0 
    END AS downgrade_ind
FROM sparkify_table
WHERE (page = 'NextSong') or (page = 'Downgrade')
""")

downgrade_indicator.createOrReplaceTempView('downgrade_table')

df_ready = spark.sql("""
SELECT
    *,
    SUM(downgrade_ind)
    OVER(PARTITION BY userId 
        ORDER BY ts DESC 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period
FROM downgrade_table
WHERE userId = 200011
""")

## Data Grouping at Month Level

In [122]:
df_ready.createOrReplaceTempView("ready_table")

In [127]:
spark.sql("""
SELECT
    userId,
    date,
    SUM(CASE WHEN page = 'NextSong' THEN 1 ELSE 0 END) AS SongCount,
    MAX(gender) AS Gender, 
    ROUND(SUM(length), 2) AS SecondsConsumed,
    SUM(downgrade_ind) AS DowngradeVisits,
    COUNT(DISTINCT period) AS periodc
FROM ready_table
GROUP BY userId, date
ORDER BY 1
""").show()

+------+----------+---------+------+---------------+---------------+-------+
|userId|      date|SongCount|Gender|SecondsConsumed|DowngradeVisits|periodc|
+------+----------+---------+------+---------------+---------------+-------+
|200011|2018-10-02|       32|     M|        7434.23|              0|      1|
|200011|2018-10-09|       49|     M|       13544.62|              0|      1|
|200011|2018-10-11|      142|     M|       37358.64|              0|      1|
|200011|2018-10-12|       99|     M|       24818.35|              0|      1|
|200011|2018-10-13|      101|     M|       25794.23|              2|      3|
|200011|2018-10-24|       20|     M|        4515.78|              1|      2|
|200011|2018-10-26|       94|     M|       23857.15|              4|      5|
|200011|2018-10-27|       54|     M|       14043.48|              1|      2|
|200011|2018-10-31|       54|     M|       13532.19|              2|      3|
|200011|2018-11-01|        5|     M|        1319.63|              0|      1|

In [133]:
df.filter((df.page == 'NextSong') | (df.page == 'Downgrade'))\
          .filter(df.userId == '200011').show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+----------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+----------+-------------------+----------+-----------+-------------+---------------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|        location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|      date|           datetime|cancel_ind|cancel_user|downgrade_ind|downgrade_phase|
+--------------------+---------+---------+------+-------------+--------+---------+-----+----------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+----------+-------------------+----------+-----------+-------------+---------------+
|Iarla ÃÂ LionÃ...|Logged In|  Charles|     M|            4|  Harper|335.90812| free|St. Louis, M

## Gender Encoding

### Flat Maps

In [92]:
df.select("gender").distinct().rdd.flatMap(lambda x: x).collect()

['F', None, 'M']

# Machine Learning

In [70]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- date: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- cancel_ind: integer (nullable = true)
 |-- cancel_user: long (nullable = true)
 |-- downgrade_ind: integer (nullable = true)
 |-- downgrade_phase: long (nullable = true)



In [73]:
df.select(["length", "page"]).show()

+---------+---------------+
|   length|           page|
+---------+---------------+
|277.89016|       NextSong|
|236.09424|       NextSong|
| 282.8273|       NextSong|
|262.71302|       NextSong|
|223.60771|       NextSong|
|208.29995|       NextSong|
|260.46649|       NextSong|
|185.44281|       NextSong|
|     null|Add to Playlist|
|134.47791|       NextSong|
| 229.8771|       NextSong|
|     null|    Roll Advert|
|223.58159|       NextSong|
|201.06404|       NextSong|
|     null|      Thumbs Up|
|246.69995|       NextSong|
|168.64608|       NextSong|
| 166.1122|       NextSong|
|222.22322|       NextSong|
|229.77261|       NextSong|
+---------+---------------+
only showing top 20 rows



In [74]:
df.select("datetime").show()

+-------------------+
|           datetime|
+-------------------+
|2018-09-30T20:01:57|
|2018-09-30T20:03:00|
|2018-09-30T20:06:34|
|2018-09-30T20:06:56|
|2018-09-30T20:11:16|
|2018-09-30T20:11:18|
|2018-09-30T20:14:46|
|2018-09-30T20:14:59|
|2018-09-30T20:15:05|
|2018-09-30T20:18:04|
|2018-09-30T20:19:06|
|2018-09-30T20:19:10|
|2018-09-30T20:20:18|
|2018-09-30T20:22:55|
|2018-09-30T20:22:56|
|2018-09-30T20:24:01|
|2018-09-30T20:26:16|
|2018-09-30T20:27:48|
|2018-09-30T20:28:07|
|2018-09-30T20:29:04|
+-------------------+
only showing top 20 rows



In [ ]:
df.groupBy('datetime')